In [42]:
df

,Loan_ID,LOAN,LOANDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,772418,1100,25860.0,39025.0,CarImp,Other,10.5,0.0,0.0,94.366667,1.0,9.0,NaN
1,477724,1300,70053.0,68400.0,CarImp,Other,7.0,0.0,2.0,121.833333,0.0,14.0,NaN
2,150746,1500,13500.0,16700.0,CarImp,Other,4.0,0.0,0.0,149.466667,1.0,10.0,NaN
3,108584,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,321534,1700,97800.0,112000.0,CarImp,Office,3.0,0.0,0.0,93.333333,0.0,14.0,NaN
5,874921,1700,30548.0,40320.0,CarImp,Other,9.0,0.0,0.0,101.466002,1.0,8.0,37.113614
6,511718,1800,48649.0,57037.0,CarImp,Other,5.0,3.0,2.0,77.100000,1.0,17.0,NaN
7,119768,1800,28502.0,43034.0,CarImp,Other,11.0,0.0,0.0,88.766030,0.0,8.0,36.884894
8,290726,2000,32700.0,46740.0,CarImp,Other,3.0,0.0,2.0,216.933333,1.0,12.0,NaN
9,245544,2000,NaN,62250.0,CarImp,Sales,16.0,0.0,0.0,115.800000,0.0,13.0,NaN


In [31]:
import pandas as pd
import numpy as np
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from datetime import date
import random

df = pd.read_csv('Data_orig/AutoLoan.csv',sep= ',')[11:]
df_scoring = pd.read_csv('Data_orig/AutoLoan.csv',sep= ',')[0:11]

df.insert(1, 'APP_DATE',pd.date_range(start='1/1/2005', periods=len(df), freq='D'))

df=df.sample(frac=1).reset_index(drop=True)

df['index']= df.index
 
def action_if(x): 
    if (x < 604):
        return 'Put More Money Down'
    elif (x < 1199):
        return 'Offer Longer Contract Term'
    elif (x < 1492):
        return 'Switch to Less Expensive Vehicle'
    elif (x < 1794):
        return 'Swith to Lease'
    elif (x < 2389):
        return 'Switch to Used Vehicle'
    elif (x < 2984):
        return 'Offer Economy Vehicle'
    elif (x < 3579):
        return 'Offer Small SUV'
    elif (x < 4175):
        return 'Offer Small Pickup Truck'
    elif (x < 4771):
        return 'Offer Sedan'
    else:
        return 'Renew Existing Lease'
    
action_group = df.copy()   
df['ACTION'] = action_group['index'].apply(action_if)

df2 = df.fillna(0)

#df3 = df2.drop(['index', 'APP_DATE', 'BAD'], axis=1)

df_history = df2.drop(['index', 'BAD'], axis=1)
 
dfgene =df2[['BAD', 'LOAN', 'LOANDUE', 'VALUE', 'YOJ', 'DEROG','DELINQ', 'CLAGE', 'NINQ', 'CLNO', 'DEBTINC']].to_numpy(dtype= 'float32')
dfgene2 =df2[['BAD', 'LOAN', 'LOANDUE', 'VALUE', 'YOJ', 'DEROG','DELINQ', 'CLAGE', 'NINQ', 'CLNO', 'DEBTINC']]
 
#write out remediation history and new failures
df_history.to_csv('/sasinside/userdata/gegrab/resources/hmeq/AutoLoan_remediation_history.csv', index=False)
 
 
#df_scoring.to_csv('Data_orig/AutoLoan_scoring.csv', index=False) Just Use Autoloan_test_1
 

In [3]:
auto_index =0
cosine_sim = cosine_similarity(dfgene)
similar_auto = list(enumerate(cosine_sim[auto_index])) 
sorted_similar_auto = sorted(similar_auto, key=lambda x:x[1], reverse=True)
sorted_similar_auto2 = pd.DataFrame(sorted(similar_auto, key=lambda x:x[1], reverse=True))

print(sorted_similar_auto[0:11])
     

[(0, 0.9999998), (4531, 0.999996), (82, 0.9999937), (5604, 0.999991), (193, 0.9999874), (1458, 0.9999752), (498, 0.9999736), (2757, 0.9999715), (5363, 0.9999708), (4963, 0.99996215), (3355, 0.99995023)]


In [4]:
# Print remediation and date of machines that had the highest cosine simularity

def get_auto_from_index2(index):
     
    return (df2[df2.index == index]["Loan_ID"].values[0], df2[df2.index == index]["ACTION"].values[0],
            df2[df2.index == index]["APP_DATE"].values[0])
            #sorted_similar_machines[i][1], sorted_similar_machines[i][0])
            
 

print('Loan ID with High Predicted Default:\n',df2[df2.index == 0]["Loan_ID"].values[0]) 
print('\n')

i=0
print('"Loan IDs with Similar Application Features and the Ensuing Response Action"')
print('Loan IDs + Response + Response Date:\n')
for auto in sorted_similar_auto:
    i=i+1
    
    if i > 1:
        print(get_auto_from_index2(auto[0]))  
    if i  > 10:
        break
    

Loan ID with High Predicted Default:
 759856


"Loan IDs with Similar Application Features and the Ensuing Response Action"
Loan IDs + Response + Response Date:

(19035, 'Offer Sedan', numpy.datetime64('2012-05-25T00:00:00.000000000'))
(972048, 'Put More Money Down', numpy.datetime64('2016-01-23T00:00:00.000000000'))
(604901, 'Renew Existing Lease', numpy.datetime64('2019-03-07T00:00:00.000000000'))
(200760, 'Put More Money Down', numpy.datetime64('2016-08-12T00:00:00.000000000'))
(362241, 'Switch to Less Expensive Vehicle', numpy.datetime64('2019-03-06T00:00:00.000000000'))
(891634, 'Put More Money Down', numpy.datetime64('2011-08-31T00:00:00.000000000'))
(333791, 'Offer Economy Vehicle', numpy.datetime64('2018-02-27T00:00:00.000000000'))
(941332, 'Renew Existing Lease', numpy.datetime64('2016-05-18T00:00:00.000000000'))
(653810, 'Renew Existing Lease', numpy.datetime64('2014-08-24T00:00:00.000000000'))
(459732, 'Offer Small SUV', numpy.datetime64('2015-09-30T00:00:00.000000000'))


In [38]:
''' List all output parameters as comma-separated values in the "Output:" docString. Do not specify "None" if there is no output parameter. '''
def execute(Loan_ID,
            LOAN,
            LOANDUE,
            VALUE,
            REASON,
            JOB,
            YOJ,
            DEROG,
            DELINQ,
            CLAGE,
            NINQ,
            CLNO,
            DEBTINC):
    "Output: COSINE_1, LOAN_ID_1, APP_DATE_1, ACTION_1"
    #"Output: gene"
             
    import pandas as pd
    import numpy as np
    import pickle
    from sklearn.metrics.pairwise import cosine_similarity
    
    inputArray = pd.DataFrame([[Loan_ID, LOAN, LOANDUE, VALUE, REASON, JOB, YOJ, DEROG, DELINQ, CLAGE, NINQ, CLNO, DEBTINC]],
                              columns = ['Loan_ID', 'LOAN','LOANDUE','VALUE','REASON','JOB','YOJ','DEROG','DELINQ','CLAGE','NINQ','CLNO','DEBTINC']
                             )
    
    inputArray2 =inputArray[['LOAN','LOANDUE','VALUE','YOJ','DEROG','DELINQ','CLAGE','NINQ','CLNO','DEBTINC']]
    inputArray2.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/autoloan_impute.pickle', 'rb')),inplace=True)
     
    df = pd.read_csv('/sasinside/userdata/gegrab/resources/hmeq/AutoLoan_remediation_history.csv',sep= ',')
    df['index']= df.index+1 
    
    df2 =df[['LOAN','LOANDUE','VALUE','YOJ','DEROG','DELINQ','CLAGE','NINQ','CLNO','DEBTINC']]
    
    combined = pd.concat([inputArray2,df2], sort=False)
    auto_index =0
    cosine_sim = cosine_similarity(combined) 
    similar_auto = list(enumerate(cosine_sim[auto_index])) 
    sorted_similar_auto = pd.DataFrame(sorted(similar_auto, key=lambda x:x[1], reverse=True))
    sorted_similar_auto.columns  = ['index', 'cosine_sim']
    
    merged = pd.merge(sorted_similar_auto, df,on=['index'])
    
    COSINE_1 = float(merged.iloc[0][1])
    
    LOAN_ID_1 = float(merged.iloc[0][2])
     
    APP_DATE_1 = merged.iloc[0][3]
    
    ACTION_1 = merged.iloc[0][16]
   
    
    return (COSINE_1, LOAN_ID_1, APP_DATE_1, ACTION_1)
     

In [40]:
import pandas as pd
df = pd.read_csv('Data_orig/AutoLoan_TEST_1.csv')
df.drop('BAD',axis=1, inplace=True)
 
 
for i in range(10):
    print(execute(**df.iloc[i]))

/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


(0.9999714729388071, 655857.0, '2006-06-11', 'Offer Small Pickup Truck')


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


(0.9995521571354261, 69613.0, '2005-05-21', 'Renew Existing Lease')


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


...

(0.9999915472369105, 149264.0, '2006-05-18', 'Offer Longer Contract Term')


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


(0.9999930393710171, 846098.0, '2005-09-05', 'Offer Small Pickup Truck')


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


(0.9985767456053145, 473967.0, '2005-02-09', 'Offer Longer Contract Term')
